In [ ]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from nn.models import OneHiddenMLP
from nn.training import train_passive, TrainConfig
from nn.evaluation import evaluate_regression
from nn.experiments import ActiveConfig, run_active_regression
from nn.strategies import uncertainty_sampling, sensitivity_sampling, UncertaintySamplingConfig

from typing import Dict

SAVE_DIR = os.path.join('..', 'report', 'figures')
DATA_DIR = os.path.join('..', 'data')
os.makedirs(SAVE_DIR, exist_ok=True)

with open(os.path.join(DATA_DIR, 'reg_uncertainty_results.json'), 'r') as f:
    unc_results = json.load(f)
with open(os.path.join(DATA_DIR, 'reg_sensitivity_results.json'), 'r') as f:
    sen_results = json.load(f)
with open(os.path.join(DATA_DIR, 'passive_reg_best.json'), 'r') as f:
    pas_results = json.load(f)

DATASETS = ['diabetes', 'wine_quality', 'california']
METRICS = ['rmse', 'mae', 'r2']
UNCERTAINTY_METHODS = ['entropy', 'margin', 'least_confidence']
ACTIVE_PARAMS = {
    'diabetes': {'init': 20, 'query': 10, 'budget': 150},
    'wine_quality': {'init': 30, 'query': 15, 'budget': 300},
    'california': {'init': 50, 'query': 20, 'budget': 1000}
}

In [ ]:
def get_data_splits_regression(dataset: str):
    # Load data
    if dataset == "diabetes":
        ds = datasets.load_diabetes()
        y = ds.target.astype(np.float32)
    elif dataset == "wine_quality":
        from sklearn.datasets import fetch_openml
        ds = fetch_openml('wine-quality-red', version=1, as_frame=False, parser='auto')
        X = ds.data.astype(np.float32)
        y = ds.target.astype(np.float32)
    elif dataset == "california":
        ds = datasets.fetch_california_housing()
        y = ds.target.astype(np.float32)
        X = ds.data.astype(np.float32)
    
    # Only set X here if it wasn't already set above
    if dataset != "wine_quality":
        X = ds.data.astype(np.float32)
    
    X_train_val, X_test, y_train_val, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    return X_train_val, X_test, y_train_val, y_test

def evaluate_passive_test_regression(dataset: str) -> Dict[str, float]:
    # Get best hyperparameters from tuning results
    best_cfg = pas_results[dataset]['best_cfg']
    lr = best_cfg['lr']
    wd = best_cfg['wd']
    hidden = best_cfg['hidden']
    bs = best_cfg['bs']
    
    # Get data splits
    X_train_val, X_test, y_train_val, y_test = get_data_splits_regression(dataset)
    
    all_metrics = []

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_val)
    X_test_scaled = scaler.transform(X_test)
    
    # Convert to tensors
    X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train_val, dtype=torch.float32).unsqueeze(-1)
    X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(-1)
    
    # Create datasets
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    
    train_loader = DataLoader(train_dataset, batch_size=bs, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=bs, shuffle=False)
    
    # Train model
    model = OneHiddenMLP(input_dim=X_train_scaled.shape[1], hidden_units=hidden, output_dim=1)
    loss_fn = nn.MSELoss()
    config = TrainConfig(learning_rate=lr, weight_decay=wd, batch_size=bs, max_epochs=200, patience=20)
    
    train_passive(model, train_loader, test_loader, loss_fn, config)
    
    # Evaluate on test set
    metrics = evaluate_regression(model, test_loader)
    
    return metrics

def evaluate_active_test_regression(dataset: str, strategy: str, method: str, budget: int) -> Dict[str, float]:
    dataset_params = ACTIVE_PARAMS[dataset]
    init = dataset_params['inits']
    query = dataset_params['queries']
    hidden, bs = 64, 64
    if strategy == 'uncertainty':
        best_cfg = unc_results[dataset][method]['best_cfg']['train_config']
        lr = best_cfg['learning_rate']
        wd = best_cfg['weight_decay']
    else:
        best_cfg = sen_results[dataset]['best_cfg']['train_config']
        lr = best_cfg['learning_rate']
        wd = best_cfg['weight_decay']

    # Get data splits
    X_train_val, X_test, y_train_val, y_test = get_data_splits_regression(dataset)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_val)
    X_test_scaled = scaler.transform(X_test)
    
    # Convert to tensors
    X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train_val, dtype=torch.float32).unsqueeze(-1)
    X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(-1)
    
    # Create datasets
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

    test_loader = DataLoader(test_dataset, batch_size=bs, shuffle=False)
    
    # Simulate active learning on the train+val set
    train_config = TrainConfig(learning_rate=lr, weight_decay=wd, batch_size=bs, 
                                max_epochs=200, patience=20)
    
    # Create initial labeled pool
    num_train = X_train_scaled.shape[0]
    labeled_indices = torch.randperm(num_train)[:init]
    unlabeled_indices = torch.tensor([i for i in range(num_train) if i not in labeled_indices.tolist()], dtype=torch.long)
    
    x_pool = torch.tensor(X_train_scaled, dtype=torch.float32)
    y_pool = y_train_tensor.clone()
    
    # Active learning loop
    while labeled_indices.numel() < min(budget, num_train):
        # Train model on current labeled set
        train_subset = TensorDataset(x_pool[labeled_indices], y_pool[labeled_indices])
        
        train_loader = DataLoader(train_subset, batch_size=bs, shuffle=True)
        
        model = OneHiddenMLP(input_dim=X_train_scaled.shape[1], hidden_units=hidden, output_dim=1)
        loss_fn = nn.MSELoss()
        
        train_passive(model, train_loader, test_loader, loss_fn, train_config)
        
        if unlabeled_indices.numel() == 0:
            break
        
        # Query selection
        if strategy == 'uncertainty':
            sel = uncertainty_sampling(
                model,
                x_pool[unlabeled_indices].to('cpu'),
                query,
                UncertaintySamplingConfig(mode="regression", method=method),
            )
        elif strategy == 'sensitivity':
            sel = sensitivity_sampling(model, x_pool[unlabeled_indices].to(train_config.device), query)
        
        # Update labeled and unlabeled sets
        newly_selected = unlabeled_indices[sel]
        labeled_indices = torch.unique(torch.cat([labeled_indices, newly_selected]))
        mask = torch.ones_like(unlabeled_indices, dtype=torch.bool)
        mask[sel] = False
        unlabeled_indices = unlabeled_indices[mask]
        
        if labeled_indices.numel() >= budget:
            break
    
    # Final evaluation on test set
    final_train_subset = TensorDataset(x_pool[labeled_indices], y_pool[labeled_indices])
    final_train_loader = DataLoader(final_train_subset, batch_size=bs, shuffle=True)
    final_test_loader = DataLoader(test_dataset, batch_size=bs, shuffle=False)
    
    final_model = OneHiddenMLP(input_dim=X_train_scaled.shape[1], hidden_units=hidden, output_dim=1)
    loss_fn = nn.MSELoss()
    
    train_passive(final_model, final_train_loader, final_test_loader, loss_fn, train_config)
    
    # Evaluate on test set
    metrics = evaluate_regression(final_model, final_test_loader, device='cpu')
    
    return metrics


In [ ]:
test_results = {
    'passive': {},
    'uncertainty': {},
    'sensitivity': {}
}

# Evaluate passive learning on test set
for dataset in DATASETS:
    print(f"Running passive on {dataset}")
    test_results['passive'][dataset] = evaluate_passive_test_regression(dataset)

# Evaluate uncertainty-based active learning on test set
for dataset in DATASETS:
    budget = ACTIVE_PARAMS[dataset]['budget']
    test_results['uncertainty'][dataset] = {}
    for method in UNCERTAINTY_METHODS:
        print(f"Running uncertainty on {dataset} - {method}")
        test_results['uncertainty'][dataset][method] = {}
        test_results['uncertainty'][dataset][method][str(budget)] = evaluate_active_test_regression(dataset, 'uncertainty', method, budget)

# Evaluate sensitivity-based active learning on test set
for dataset in DATASETS:
    budget = ACTIVE_PARAMS[dataset]['budget']
    print(f"Running sensitivity on {dataset}...")
    test_results['sensitivity'][dataset] = {}
    test_results['sensitivity'][dataset][str(budget)] = evaluate_active_test_regression(dataset, 'sensitivity', '', budget)

print("Done")

In [ ]:
for dataset in DATASETS:
    b = ACTIVE_PARAMS[dataset]['budget']
    for metric in METRICS:

        plt.figure(figsize=(10, 6))
        
        # Plot uncertainty methods
        for method in UNCERTAINTY_METHODS:
            result = [test_results['uncertainty'][dataset][method][str(b)][f'{metric}']]
        
        # Plot sensitivity method
        result = [test_results['sensitivity'][dataset][str(b)][f'{metric}']]
        
        # Plot passive baseline as horizontal line
        baseline = test_results['passive'][dataset][f'{metric}']
        plt.axhline(baseline, color='k', linestyle='--', label='passive_best', linewidth=2)
        
        plt.xlabel('Labeled budget (max_labels)')
        plt.ylabel(metric)
        plt.title(f'{dataset}: Strategies Comparison ({metric}) - TEST SET EVALUATION')
        plt.grid(True, alpha=0.3)
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.tight_layout()
        plt.savefig(os.path.join(SAVE_DIR, f'reg_{dataset}_comparison_{metric}_test.png'), dpi=200, bbox_inches='tight')
        plt.close()
        
        print(f"Saved {dataset} {metric} comparison (TEST SET)")


In [ ]:
summary_data = []

for dataset in DATASETS:

    budget = ACTIVE_PARAMS[dataset]['budget']
    # Passive learning
    summary_data.append({
        'dataset': dataset,
        'method': 'passive',
        'budget': budget,
        'rmse': test_results['passive'][dataset]['rmse'],
        'mae': test_results['passive'][dataset]['mae'],
        'r2': test_results['passive'][dataset]['r2'],
    })
    
    # Uncertainty methods
    for method in UNCERTAINTY_METHODS:
        max_budget = str(budget)
        summary_data.append({
            'dataset': dataset,
            'method': f'uncertainty_{method}',
            'budget': budget,
            'rmse': test_results['uncertainty'][dataset][method][max_budget]['rmse'],
            'mae': test_results['uncertainty'][dataset][method][max_budget]['mae'],
            'r2': test_results['uncertainty'][dataset][method][max_budget]['r2'],
        })
    
    # Sensitivity method
    max_budget = str(budget)
    summary_data.append({
        'dataset': dataset,
        'method': 'sensitivity',
        'budget': budget,
        'rmse': test_results['sensitivity'][dataset][max_budget]['rmse'],
        'mae': test_results['sensitivity'][dataset][max_budget]['mae'],
        'r2': test_results['sensitivity'][dataset][max_budget]['r2'],
    })

# Convert to DataFrame for nice display
df = pd.DataFrame(summary_data)
print(df.round(4))

# Save summary
df.to_csv(os.path.join(SAVE_DIR, 'reg_comparison_summary_test.csv'), index=False)